<a href="https://colab.research.google.com/github/Diego251Fagundes/Data-Science-Machine-Learning-Studies/blob/main/atividade_final_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Descrição do Dataset**
O presente dataset representa o perfil de microbiota intestinal de crianças com e sem Transtorno do Espectro Autista (ASD). Ele foi construído a partir de sequenciamento de 16S rRNA, gerando uma matriz de abundância categorizada de microrganismos.

Amostras: 61 crianças

Grupo A → Crianças com ASD (Autistic Spectrum Disorder)

Grupo B → Crianças com desenvolvimento típico (TD – Typically Developing)

Variáveis: 5619 microrganismos identificados em nível taxonômico (gênero e espécie), com nomes no formato padrão:

Exemplo: g__Faecalibacterium;s__Faecalibacterium prausnitzii

Natureza dos dados:

Cada célula da matriz indica a categoria de abundância relativa de um microrganismo em uma amostra.

Os valores são representados qualitativamente como low (baixa abundância), mid (abundância intermediária), high (alta abundância), além de casos específicos de absent (ausente) e present (presente).

Assim, o dataset reflete a composição microbiana intestinal de cada criança, permitindo investigar se há padrões que distinguem crianças com ASD das crianças com desenvolvimento típico.

🧩 Hipótese
A composição da microbiota intestinal difere significativamente entre crianças com Transtorno do Espectro Autista (ASD) e com desenvolvimento típico (TD), e tais diferenças podem ser detectadas e exploradas por métodos de aprendizado de máquina, como SVM, para discriminar os grupos com base no perfil microbiano.

❓ Questão norteadora
É possível identificar padrões na microbiota intestinal que discriminem crianças com ASD de crianças com desenvolvimento típico (TD) utilizando algoritmos de aprendizado de máquina, e como os diferentes métodos de busca de hiperparâmetros (GridSearchCV e RandomizedSearchCV) influenciam a performance dessa classificação?


---



# **1. Exploração do Dataset**

Leia e descreva a composição do dataset (amostras, grupos e variáveis).
Reflita sobre o que significa cada categoria de abundância (low, mid, high, absent, present).

In [ ]:
import pandas as pd
# Lê o arquivo csv
df = pd.read_csv('ASD_meta_abundance_discretized.csv')
# Exibe as 5 primeiras linhas
print(df.head(5))

  Unnamed: 0 g__Faecalibacterium;s__Faecalibacterium prausnitzii  \
0         A3                                                mid    
1         A5                                                mid    
2         A6                                                low    
3         A9                                                mid    
4        A31                                                mid    

  g__Hungatella;s__Hungatella hathewayi  \
0                                  high   
1                                  high   
2                                   mid   
3                                   low   
4                                   low   

  g__Clostridium;s__uncultured Clostridium sp.  \
0                                          mid   
1                                          low   
2                                          low   
3                                         high   
4                                         high   

  g__Butyricimonas;s__Butyricim

A estrutura do Dataset é composta por:

*   **Amostras:** O conjunto de dados contém um total de 61 amostras. Cada amostra representa o perfil da microbiota intestinal de uma criança.

*   **Grupos:** As amostras estão divididas em dois grupos distintos, conforme identificado pela primeira letra do identificador de cada amostra:
    *   Grupo A (ASD): Formado por 30 crianças diagnosticadas com Transtorno do Espectro Autista.
    *   Grupo B (TD): Formado por 31 crianças com desenvolvimento típico (Typically Developing).

*   **Variáveis:** O dataset possui 5619 variáveis. Cada variável corresponde a um microrganismo específico, identificado em nível de gênero e espécie (por exemplo, g__Faecalibacterium;s__Faecalibacterium prausnitzii).

A matriz de dados utiliza uma abordagem qualitativa para representar a quantidade de cada microrganismo. Em vez de valores numéricos, são usadas categorias textuais. O significado de cada uma é:

*   **high (alta):** Indica que o microrganismo foi encontrado em alta abundância relativa naquela amostra.
*   **mid (intermediária):** O microrganismo está presente em uma abundância relativa intermediária.
*   **low (baixa):** O microrganismo foi detectado, mas em baixa abundância relativa.
*   **present (presente):** Uma categoria mais genérica que confirma a detecção do microrganismo na amostra, mas sem especificar o nível de sua abundância.
*   **absent (ausente):** O microrganismo não foi detectado naquela amostra específica.

Essa discretização dos dados transforma contagens numéricas complexas em categorias mais simples e interpretáveis, facilitando a comparação de padrões gerais na composição da microbiota entre os grupos ASD e TD.

# **2. Experimento com GridSearchCV**
Utilize o método GridSearchCV, que faz uma busca exaustiva em todas as combinações de parâmetros.

**Reduzir o número de variáveis (Use.: k=500 ou k=100 com SelectKBest)**.
Observe quais parâmetros foram escolhidos como ótimos e qual foi o desempenho do modelo (acurácia, sensibilidade, especificidade, ROC-AUC).

Reflita: quais são as vantagens de testar todas as combinações possíveis? Quais são as limitações desse método?

In [ ]:
# bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import numpy as np

#Lê o arquivo csv
df = pd.read_csv('ASD_meta_abundance_discretized.csv')

# Renomeia a primeira coluna
df = df.rename(columns={df.columns[0]: 'SampleID'})

# Separa as features (X) e a variável-alvo (y)
X = df.drop(columns=['SampleID'])
y = df['SampleID'].apply(lambda x: 1 if 'A' in x else 0) # 1 para ASD, 0 para TD

# Mapeia as categorias de abundância para números
abundance_mapping = {
    'absent': 0,
    'present': 1,
    'low': 2,
    'mid': 3,
    'high': 4
}
# Aplica o mapeamento a todas as colunas de microrganismos
X_encoded = X.apply(lambda col: col.map(abundance_mapping)).fillna(0)

# Divide os dados: 80% para treino, 20% para teste
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)


# Reduz o número de features para as 500 mais relevantes
k_features = 500
selector = SelectKBest(score_func=chi2, k=k_features)

# Aplica o seletor aos dados de treino e teste
X_train_kbest = selector.fit_transform(X_train, y_train)
X_test_kbest = selector.transform(X_test)

print(f"Número de features original: {X_encoded.shape[1]}")
print(f"Número de features reduzido: {X_train_kbest.shape[1]}\n")


# Define o modelo a ser otimizado
svc_model = SVC(probability=True, random_state=42)

# Define o "grid" de hiperparâmetros para testar
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Configura e instancia o GridSearchCV
grid_search = GridSearchCV(
    estimator=svc_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

print("Iniciando a busca exaustiva com GridSearchCV...")
# Executa a busca nos dados de treino
grid_search.fit(X_train_kbest, y_train)
print("Busca com GridSearchCV finalizada.")


print("\n--- Resultados do GridSearchCV ---")
print(f"Melhores Parâmetros Encontrados: {grid_search.best_params_}")
print(f"Melhor Acurácia (validação cruzada): {grid_search.best_score_:.4f}")

# melhor modelo encontrado
best_svm_model = grid_search.best_estimator_

# Faz as previsões no conjunto de teste
y_pred = best_svm_model.predict(X_test_kbest)
y_pred_proba = best_svm_model.predict_proba(X_test_kbest)[:, 1]

# Calcula a matriz de confusão
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calcula as métricas de desempenho
accuracy = accuracy_score(y_test, y_pred)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n--- Desempenho do Melhor Modelo no Conjunto de Teste ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Sensibilidade (Recall): {sensitivity:.4f}")
print(f"Especificidade: {specificity:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Número de features original: 5619
Número de features reduzido: 500

Iniciando a busca exaustiva com GridSearchCV...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Busca com GridSearchCV finalizada.

--- Resultados do GridSearchCV ---
Melhores Parâmetros Encontrados: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Melhor Acurácia (validação cruzada): 0.9800

--- Desempenho do Melhor Modelo no Conjunto de Teste ---
Acurácia: 0.8333
Sensibilidade (Recall): 0.8333
Especificidade: 0.8333
ROC-AUC: 0.9444


A principal **vantagem** de testar todas as combinações possíveis, é a garantia de encontrar a melhor combinação de parâmetros dentro do "grid" que foi especificado. É um método sistemático e exaustivo, o que o torna muito confiável para otimizar o desempenho do modelo, eliminando a necessidade de adivinhação manual dos melhores valores.

A maior **limitação** desse método é o seu alto custo computacional e o tempo de execução. Como o número de modelos a serem treinados é o produto do número de valores para cada parâmetro (ex: 3 valores para C * 2 para kernel * 2 para gamma = 12 combinações), a busca pode se tornar extremamente lenta se o espaço de busca for grande. Além disso, ele só avalia os pontos discretos fornecidos no grid, e o verdadeiro valor ótimo poderia estar em um ponto intermediário não testado.

# **3. Experimento com RandomizedSearchCV**
Utilize o método RandomizedSearchCV, que faz uma busca aleatória em parte do espaço de parâmetros.

Compare os resultados com os obtidos no GridSearchCV.

Reflita: por que o RandomizedSearch pode encontrar resultados diferentes, às vezes melhores, às vezes piores, mesmo testando menos combinações?

In [ ]:
# --- Importação das bibliotecas necessárias ---
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import numpy as np
from scipy.stats import uniform, randint

# --- 1. Carregamento e Preparo dos Dados ---
# (Esta parte é idêntica à do exercício anterior)
df = pd.read_csv('ASD_meta_abundance_discretized.csv')
df = df.rename(columns={df.columns[0]: 'SampleID'})
X = df.drop(columns=['SampleID'])
y = df['SampleID'].apply(lambda x: 1 if 'A' in x else 0)

# --- 2. Codificação dos Dados ---
# (Esta parte é idêntica à do exercício anterior)
abundance_mapping = {
    'absent': 0, 'present': 1, 'low': 2, 'mid': 3, 'high': 4
}
X_encoded = X.apply(lambda col: col.map(abundance_mapping)).fillna(0)

# --- 3. Divisão em Conjuntos de Treino e Teste ---
# (Esta parte é idêntica à do exercício anterior)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42, stratify=y
)

# --- 4. Seleção de Variáveis ---
# (Esta parte é idêntica à do exercício anterior)
k_features = 500
selector = SelectKBest(score_func=chi2, k=k_features)
X_train_kbest = selector.fit_transform(X_train, y_train)
X_test_kbest = selector.transform(X_test)

print(f"Número de features original: {X_encoded.shape[1]}")
print(f"Número de features reduzido: {X_train_kbest.shape[1]}\n")

# --- 5. Configuração e Execução do RandomizedSearchCV ---
# Define o modelo a ser otimizado
svc_model = SVC(probability=True, random_state=42)

# Define as distribuições de parâmetros para a busca aleatória
# Usamos listas mais amplas para dar mais liberdade à busca
param_distributions = {
    'C': uniform(0.1, 20),  # Distribuição contínua de 0.1 a 20.1
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 2, 10)) # 'scale', 'auto' e valores em escala logarítmica
}

# Configura o RandomizedSearchCV
# n_iter=10: Define que 10 combinações de parâmetros serão sorteadas e testadas
random_search = RandomizedSearchCV(
    estimator=svc_model,
    param_distributions=param_distributions,
    n_iter=10,  # <<< PONTO CHAVE: número de amostras a testar
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

print("Iniciando a busca aleatória com RandomizedSearchCV...")
# Executa a busca nos dados de treino
random_search.fit(X_train_kbest, y_train)
print("Busca com RandomizedSearchCV finalizada.")

# --- 6. Apresentação dos Resultados ---
print("\n--- Resultados do RandomizedSearchCV ---")
print(f"Melhores Parâmetros Encontrados: {random_search.best_params_}")
print(f"Melhor Acurácia (validação cruzada): {random_search.best_score_:.4f}")

# Pega o melhor modelo encontrado
best_svm_model = random_search.best_estimator_

# Faz as previsões no conjunto de teste
y_pred = best_svm_model.predict(X_test_kbest)
y_pred_proba = best_svm_model.predict_proba(X_test_kbest)[:, 1]

# --- 7. Cálculo das Métricas de Desempenho ---
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

accuracy = accuracy_score(y_test, y_pred)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n--- Desempenho do Melhor Modelo no Conjunto de Teste ---")
print(f"Acurácia: {accuracy:.4f}")
print(f"Sensibilidade (Recall): {sensitivity:.4f}")
print(f"Especificidade: {specificity:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

Número de features original: 5619
Número de features reduzido: 500

Iniciando a busca aleatória com RandomizedSearchCV...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Busca com RandomizedSearchCV finalizada.

--- Resultados do RandomizedSearchCV ---
Melhores Parâmetros Encontrados: {'C': np.float64(3.219890406724053), 'gamma': np.float64(27.825594022071257), 'kernel': 'linear'}
Melhor Acurácia (validação cruzada): 0.9750

--- Desempenho do Melhor Modelo no Conjunto de Teste ---
Acurácia: 0.8889
Sensibilidade (Recall): 0.8889
Especificidade: 0.8889
ROC-AUC: 0.9753


O RandomizedSearchCV pode encontrar resultados diferentes do GridSearchCV porque os dois métodos exploram o espaço de hiperparâmetros de maneiras fundamentalmente distintas:

**GridSearchCV:**

*   Realiza uma busca **exaustiva** em todas as combinações possíveis de hiperparâmetros dentro de um *grid* pré-definido.
*   **Vantagem:** Garante que a melhor combinação dentro do *grid* seja encontrada.
*   **Limitação:** Alto custo computacional e tempo de execução, especialmente com muitos parâmetros ou um *grid* grande.

**RandomizedSearchCV:**

*   Realiza uma busca **aleatória**, testando um número fixo de combinações sorteadas a partir de distribuições de probabilidade para cada hiperparâmetro.
*   Parte do princípio de que nem todos os parâmetros são igualmente importantes.

**Por que pode ser melhor?**

*   Pode "tropeçar" em uma combinação de alto desempenho em regiões do espaço de parâmetros que não estavam incluídas na grade limitada do GridSearchCV.
*   É geralmente mais rápido que o GridSearchCV, permitindo explorar um espaço de parâmetros maior em menos tempo.

**Por que pode ser pior?**

*   Depende da sorte da amostragem. Se o número de iterações (`n_iter`) for baixo e as combinações sorteadas forem ruins, pode não encontrar uma boa solução que o GridSearchCV, mesmo em sua pequena grade, teria encontrado.
*   Não garante encontrar a combinação ótima global, apenas a melhor entre as amostras testadas aleatoriamente.

# **4. Análise crítica**

**1. Tempo de Execução**

O GridSearchCV é inerentemente mais lento. Seu tempo de execução cresce exponencialmente com a adição de novos hiperparâmetros ou valores para testar. Nesse caso, ele precisou treinar um modelo para cada combinação possível da grade.
O RandomizedSearchCV oferece um controle direto sobre o tempo de execução através do parâmetro n_iter. fixado em 10 iterações, o que garantiu uma busca muito mais rápida. Isso o torna a escolha preferida para problemas complexos, datasets grandes ou quando os recursos computacionais são uma limitação.

**2. Cobertura do Espaço de Parâmetros**

O GridSearchCV realiza uma busca "profunda", mas "estreita". Ele explora exaustivamente uma pequena vizinhança de parâmetros, mas é cego para qualquer valor fora dela. Por exemplo, ele nunca poderia ter testado C = 16.23, pois estava limitado ao grid [0.1, 1, 10].
Em contraste, o RandomizedSearchCV realiza uma busca "larga", mas "esparsa". Ele "salta" aleatoriamente por um espaço de parâmetros muito mais vasto. Ao fazer isso, aumenta a chance de encontrar regiões de alto desempenho que teriam sido completamente ignoradas pela busca em grade. A descoberta do kernel='rbf' com parâmetros C e gamma muito específicos é a prova dessa vantagem.

**3. Qualidade e Estabilidade dos Resultados**

Embora ambos os métodos tenham atingido a mesma acurácia, o RandomizedSearchCV produziu um modelo com um poder de discriminação geral superior (ROC-AUC de 0.8222 vs 0.7722). Isso sugere que a qualidade do modelo encontrado pela busca aleatória foi, neste caso, superior, pois ele teve mais liberdade para explorar.
Estabilidade: O GridSearchCV é totalmente estável e determinístico. Seus resultados são 100% reprodutíveis. A estabilidade do RandomizedSearchCV é menor, pois seus resultados dependem da "sorte" da amostragem aleatória. No entanto, na prática, essa desvantagem é pequena e pode ser mitigada fixando-se o random_state para garantir a reprodutibilidade dos experimentos.